# A lesson on parity, Hamming codes, Reed-Solomon code, checksums, CRCs, hashes, content-addressable filesystems, git, blockchain, cryptocurrencies

This notebook will introduce you to some interesting concepts and will lead to a discussion of blockchains and cryptocurrency. There are applications in many areas of physics and computation, including transmitting data from spacecraft, and quantum computing.

The discussion is motivated by practical problems such as:

* suppose that you wish to transfer data to/from a spacecraft at the edge of the solar system, how can you do this reliably in the presence of noise?
* how can you store data reliably on a medium that has a non-zero error rate?
* practical quantum computers are not perfect devices; what can be done to correct errors in their calculations?
* how can you design a cryptocurrency system that is immune to being tampered with?

![NASA](http://mcba11.phys.unsw.edu.au/~mcba/phys3112/nh-transmission10-25-16.jpg)

## Parity bits

We start by considering how we can transmit, or store/retrieve, a binary data stream (an arbitrary sequence of bits) without corruption. A naive approach might be to transmit the stream twice, and if both copies are identical, we can have some degree of confidence that no error has occurred. To save communications bandwidth, rather than transmit two copies of the data stream, we can reduce the overhead by being clever about the additional information we send. The absolute minimal amount of information would be a single bit added to the end of the data stream that would tell you whether the sum of the data bits was an even or odd number. This would catch some types of errors (e.g., a 0 being received as a 1, or vice versa), but not others (e.g., an even number of bit errors, or transposed bits). Still, in some circumstances a single added bit may be sufficient - for example, if the chance of a single bit error in a byte is one in $10^{8}$, then the chance of a two-bit error is one in $10^{16}$, which may well be so unlikely as to be negligible.

A bit added to check the data integrity is called a parity bit. Rather than using a single parity bit at the end of the data stream, it is more usual to add a parity bit after each block of data. A common choice of block length is 8 bits, otherwise known as a byte. So, your data stream is divided into bytes, with each byte suffixed with a parity bit. Parity bits can be "even parity" or "odd parity": even parity means that the sum of all the data bits and the parity bit is an even number.

Parity bits are easily calculated using simple hardware (e.g., XOR logic gates), which makes them suitable for some applications. The next step up in complexity from a parity bit is a Hamming code.

## Hamming codes

The idea behind Hamming codes is to enable both the detection and correction of bit errors. Similarly to a parity bit, a Hamming code consists of extra bits appended to each block of data, where the number of bits added, the length of a block, and the particular type of Hamming code, are all subject to choice.

While parity bits allow the detection of an odd number of bit errors, they give no clue as to which bit(s) are incorrect. Hamming codes improve on this situation by not only detecting that an error has occurred, but pin-pointing which bit was incorrect, thereby allowing the error to be corrected without the data stream having to be retransmitted. As a simple example, suppose that you have 7 bits of data; a 3-bit Hamming code gives you $2^3$ different distinct numbers, which can be used to indicate whether an error has occurred, and which of the 7 data bits is incorrect. Hamming codes are available in various lengths, and can detect/correct various numbers of bit errors. You generally select the Hamming code to use based on the likely distribution of bit errors in your data stream, and a tradeoff between the bandwidth usage and the error rate. Hamming codes are used in ECC (error-correcting code) computer memory - this is a type of random access memory that can detect/correct bit errors. Most computer memory is not ECC, and bit errors are not detected, which can result in corruption of data and crashing of the operating system. ECC memory is used in higher-end computers, and particularly in space applications where cosmic particle flux leads to higher error rates.

If you need a higher level of error detection/correction than Hamming codes, you might consider Reed-Solomon codes.

## Reed-Solomon codes

We won't get into the details of how Reed-Solomon codes work, but the basic idea is to add even more additional bits to the data stream than Hamming codes do in order to improve the ability to detect and correct errors. Reed-Solomon codes are commonly used in storage media such as CD-ROMs, DVDs, and rotating hard disks, where media defects can cause contiguous sections of a data stream to be missing or corrupted. In a CD-ROM, for example, scratches or dirt can make some parts of the data unreadable. With Reed-Solomon codes it is possible to drill a 6-mm hole through a CD-ROM and still be able to reconstruct the data on the disk with no errors.

There is an interesting open-source program called "par2" that will accept a data stream as input and produce a stream with Reed-Solomon codes added, with a user-selectable percentage increase in total data size. "par2" is able to correct for quite large amounts of missing, duplicated, or out-of-order data blocks. Michael Ashley had an interesting application for "par2" in one of his Antarctic experiments; the situation was that a USB-attached hard disk on a scientific experiment had a faulty controller than caused a few percent of disk reads/writes to go to the wrong physical location on the disk. Needless to say, this was catastrophic to the integrity of the filesystem that was stored on the disk. 

ASIDE: a "filesystem" is the way in which a computer operating system stores data (files) and metadata (things such as file creation date, access permissions) on a physical disk. A filesystem is quite complex since it has to be able to efficiently handle very large files, large numbers of small files, file hierachies (directories/sub-directories, sometimes called folders/sub-folders), links between files, moving/deleting/renaming files, updating metadata, and possibly keeping snapshots of earlier versions of files. 

The problem with the faulty controller was only discovered after the experiment was deployed, and after everyone had left the summer-only Antarctic station for the year. The solution was to bypass using the operating system's filesystem for storing data on the disk, and instead add redundancy to the data with "par2" and store the expanded data stream sequentially by direct writes to the disk. When the disk was brought back to UNSW at the end of the year, while there were many cases of data being written to the wrong location on the disk, "par2" was able to detect and correct this, with the result that no data were lost.

## Shannon limit

In 1948 Claude Shannon considered the general question of how much data can you push through a noisy channel with no errors. I.e., if your communications system has a certain error rate, how much redundancy (in terms of adding additional check data along the lines of Hamming codes, Reed-Solomon codes, etc) is needed to recover a perfect copy of the data. Shannon's limit, or Shannon's theorem, or the noisy channel coding theorem, gives a numerical answer to this question, although it doesn't give a prescription for the algorithm for generating the extra checking data. In general, the complexity of the checking algorithm becomes more complex as you get closer to Shannon's limit. So there is a tradeoff between software complexity (and time required to do the error detection/correction) and excess bandwidth above the Shannon limit, and all of this is a function of the noise level.

## Checksums

One simple approach to data integrity is to simply sum all the bytes in a data stream, and append the result (truncated to a specified number of bytes). E.g., you might simply form an 8-bit or 16-bit sum of all the data bits. This is the approach taken by the venerable UNIX utility "sum". E.g.,

```
echo hello | sum -s
542 1
```

will display the 16-bit truncated sum (542) of all the bytes in the data stream "hello" (when converts into ASCII bytes). The number "1" is the length of the data stream in units of 512 bytes (rounded up).

This particular checksum is not very robust, since, e.g., reordering the bytes in "hello" has no effect on the sum.

```
echo elhol|sum -s
542 1
```

The BSD algorithm (using "sum -r") is slightly better, and will detect changes in byte order. The "cksum" program is even better - it uses a 32-bit cyclic redundancy check (see the next section).

## Cyclic Redundancy Checks (CRCs)

CRCs are data checksums based on passing each byte of a data stream into a polynomial function. They come in various lengths (8, 16, 32-bits are common) and can use a large variety of polynomials. See https://en.wikipedia.org/wiki/Cyclic_redundancy_check for more information.

CRCs are widely used for data integrity in communication (e.g., local area network packets, mobile phone packets, automobile busses). Each choice of bit length and polynomial has particular characteristics, and you may wish to select a CRC that is optimal for a particular application.

CRCs are good for checking for the accidental corruption of data, but they are not cryptographically secure, i.e., a determined person could construct an alternative data stream that produces any desired CRC.

## Hash functions

Parity bits, Hamming codes, Reed-Solomon codes, checksums, and CRCs are all examples of a more general concept called a hash. A hash is simply a string of binary bits resulting from applying a "hash function" to some input data. Usually, a given hash function will produce a fixed number of output bits and can operate on an arbitary amount of input data. Usually, the length of the hash is smaller than the length of the input data.

Hash functions are chosen depending on the application. For checking data integrity, the hash function is usually carefully chosen so that:

1. The output hash value is very sensitive to any changes in the input data. So, e.g., you won't get the same hash if you swap two bytes, or add/subtract an extra byte.

2. There is no known mathematical way to invert the hash function. I.e., if you have a desired hash output, then you can't come up with input data that will produce the hash, except by a laborious process of trial and error.

These properties of hashes make them quite useful for checking that data streams (or documents, files, lists of currency transactions, etc) are highly likely to have been received correctly. You might think that a better way of ensuring exact agreement would be to send a data stream twice, but this can have problems: the transmission mechanism may cause the same corruption each time (e.g., infrared communication as used for remote controls can only transmit a small number of consecutive 1 bits before it fails to reproduce them), or someone may deliberately alter both copies.

Let's look at some example hash functions.

## md5

md5 is a hash function developed in the early 1990s. It takes arbitrary input data and computes a 128-bit (32 hexadecimal digits) hash. You can experiment with it on a Linux computer using commands such as the following:

```
echo Hello | ./md5sum
```

which will produce the output:

```
09f7e02f1290be211da707a266f153b3
```

If you try changing the input "Hello", you will soon get a feeling for how difficult it is to produce any given output hash. E.g., try to produce a hash with 4 leading hexadecimal zeroes (the relevance of this example will become evident shortly).

An interesting question is: given that a hash has a fixed size, how can it possibly uniquely identify arbitrary input data? The answer is that it can't. In fact, there are an infinite number of input data streams that will generate an identical hash. The power of the hash is that it is not easy to find such "collisions", and if you have good reasons to believe that no one is deliberately investing a huge effort to do so, then you can be highly confident that your data are OK.

In the 20 years after md5's creation, a number of shortcomings were found in the algorithm, and it is now trivially easy to find collisions (different datasets that produce a desired hash).

## SHA-1 secure hash algorithm 1

SHA-1 is a hash function developed in the mid-1990s. It takes arbitrary input data and computes a 160-bit (40 hexadecimal digits) hash. You can experiment with it on a Linux computer using commands such as the following:

```
echo Hello | ./sha1sum
```

which will produce the output:

```
1d229271928d3f9e2bb0375bd6ce5db6c6d348d9 
```

## SHA-2 and SHA-3

SHA-1 has been shown to have weaknesses, so it is recommended to use the newer algorithms SHA-2 and SHA-3. SHA-2 is a set of hash functions with various bit lengths, e.g., SHA-256, SHA512. SHA-3 is the latest (2015) set of hash functions, and includes SHA-3-256 and SHA-3-512.

## git

"git", from which "github" is derived, is a "revision control system" (i.e., it keeps track of files, and the history of modifications to the files) that is based on a "content addressable filesystem". I.e., to identify a file in the filesystem, you don't use the file's name, you use its contents. Well, not exactly. The SHA-1 checksum of the file's contents are used as a proxy for the file's contents. The use of SHA-1 makes git extremely fast at determining which files have been modified when you compare your local copy of a file repository with a version stored in git.

## Cryptocurrencies

A cryptocurrency is basically a virtual alternative to physical currency (such as bank notes, coins, gold). Without a physical thing associated with the currency, it is obviously of crucial importance to have a reliable means of keeping track of transactions. This is done in a ledger, which is just a list of transactions. Traditionally a ledger is kept by a trusted third party, such as a bank. But if you don't trust a bank, how can you guarantee that the ledger is correct? You could distribute the ledger to multiple banks, and have some sort of voting system to resolve discrepancies, but this is still open to abuse and lack of trust. Blockchain is a way of creating a publicly verifiable ledger that has strong guarantees of reliability and immutability.

To see how it works, suppose that you want to store the transaction:

```
Bob gave Alice 1.27 compucoins on 25 May 2018 at 10:47:01.12am.
```

To guard against storage errors we could add a SHA-256 checksum, which for the above data (including an end-of-line character) is:

```
be2b27826c51daa9969648720dcba470ecc292e7365f8787effd76ef4f0bee89
```

We could then distribute the transaction and the checksum to a publically accessible ledger. The SHA-256 checksum guards against any modification of the transaction. However, nothing stops someone else from modifying the transaction, creating a new checksum, and uploading it to the ledger.

So the next step is to add an additional 32-bit number (called a "nonce", which is a term used in cryptography, derived from the phrase "a number used only once") to the transaction, so that the SHA-256 checksum of the transaction and the nonce has a specified number of leading zeroes. So, our transaction may look this this in the ledger:

```
Data: Bob gave Alice 1.27 compucoins on 25 May 2018 at 10:47:01.12am.
Nonce: 0x3982bd43
Checksum: 000000000000000016d6487f0d5ba670e8c292e7065f37272ffd465f4ffbe65e
```

where "0x3982bd43" is the 32-bit nonce, chosen so that the SHA-256 checksum has 16 hexadecimal digits of zeroes at the beginning (note: this particular number doesn't work). The key thing about this transaction is that finding the nonce that will produce the 16 zeroes is exceedingly difficult. It may take years on a standard desktop computer. So, if you wanted to alter the transaction to, e.g., give Alice 127 compucoins rather than 1.27, you would have to invest a lot of resources to find a nonce that would produce a suitable checksum.

The process of finding the nonce is known as "mining", and is an example of "proof of work", i.e., it proves that you have, on average, invested a certain about of effort into solving the problem. Once you have found the nonce, it is trivial for anyone to calculate the checksum and verify that you are correct. 

The combination of some transaction data, a nonce, and a checksum, is called a "block". The next step is a "blockchain", which simply adds the checksum of the previous block as part of the data of the new block. So, you might have three blocks as follows, in time sequential order:

```
Block number: 123
Data: Jane gave Fred 3.43 compucoins on 25 May 2018 at 10:46:09.12am.
Prev checksum: 000000000000000022164874044baf3028b2c2e606efaa2786fd5a5f373badc0
Nonce: 0x3982bd43
This checksum: 0000000000000000f6064874045ba23028c252e6065ff7278ffd565f3f3b3222

Block number: 124
Data: Bob gave Alice 1.27 compucoins on 25 May 2018 at 10:47:01.12am.
Prev checksum: 0000000000000000f6064874045ba23028c252e6065ff7278ffd565f3f3b3222
Nonce: 0x1233bdee
This checksum: 000000000000000016d6487f0d5ba670e8c292e7065f37272ffd465f4ffbe65e

Block number: 125
Data: John gave Sam 9.90 compucoins on 25 May 2018 at 10:47:21.12am.
Prev checksum: 000000000000000016d6487f0d5ba670e8c292e7065f37272ffd465f4ffbe65e
Nonce: 0x9374fedb
This checksum: 0000000000000000e6d5484f3d5ba6c0e8d29237065f07282f7d425f4f4b4644
```

Note how each block incorporates the checksum of the previous block. The blocks form a chain, hence "blockchain". The strength of the chain is that if you attempt to modify any block, you will have to modify not only that block, but every subsequent block, and each modification involves a very large expenditure of resources. In practice, the ledger becomes almost immutable.

What I have described above is basically how the bitcoin blockchain works. There are a few differences (e.g, multiple transactions are stored per block, transactions are stored in a data structure called a "Merkle tree", a block has a timestamp), but the core idea is identical.

The bitcoin ledger (recording all transactions ever made of bitcoins) is currently over 150 GB in size, and can be accessed by anyone. New blocks of transactions are added roughly every 10 minutes. Anyone can set up a computer to act as a "node" to receive new transactions, and if you are successful in guessing the nonce to form a new block, then you will receive 12.5 bitcoins (about $125,000 as of 25 May 2018).

The computational difficult of mining for bitcoins has spurred the development of special purpose GPUs and software. Globally, a very large amount of electricity is used for bitcoin mining. The current (25 May 2018) estimate is 70 TWh per annum, which is similar to a medium-sized country. Each bitcoin transaction requires almost a megawatt for an hour, and is responsible for 0.5 tonne of CO2 emmisions. 

## Further reading

[Here is a very nice youtube](https://www.youtube.com/watch?v=_160oMzblY8) introduction to blockchains.
